In [161]:
import pandas as pd
import numpy as np
import time
import requests
import execjs

In [447]:
# 读取基金文件
manager_history_pd = pd.read_csv('./data/manager/manager_history_pd.csv',low_memory=False)
manager_current_pd = pd.read_csv('./data/manager/manager_current_pd.csv',low_memory=False)
allManager_pd = pd.read_csv('./data/manager/allManager_pd.csv',low_memory=False)


In [448]:
# 计算经理擅长的类型，并保存
allManager_id_s = allManager_pd['manageID']
m_id = []
m_gat = []
for manage_id in allManager_id_s:
    fund_type_history_s = manager_history_pd.loc[manager_history_pd['经理ID'] ==
                                                 manage_id]['基金类型']
    fund_type_current_s = manager_current_pd.loc[manager_current_pd['经理ID'] ==
                                                 manage_id]['基金类型']
    # 合并类型
    fund_type_all_s = fund_type_history_s.append(fund_type_current_s)
#     print(manage_id,fund_type_all_s.shape)
    # 获取类型最多的，作为擅长类型
    if len(fund_type_all_s)  > 0:
        manager_goo_at_type = fund_type_all_s.groupby(
            fund_type_all_s).count().idxmax()
        m_id.append(manage_id)
        m_gat.append(manager_goo_at_type)

manager_goo_at_type_pd = pd.DataFrame(columns=['manageID', 'good_at_type'])
manager_goo_at_type_pd['manageID'] = m_id
manager_goo_at_type_pd['good_at_type'] = m_gat
allManager_good_at_type_pd = pd.merge(
    allManager_pd, manager_goo_at_type_pd, how='left', on='manageID')
allManager_good_at_type_pd.to_csv(
    './data/manager/allManager_pd.csv',
    encoding='utf_8_sig',
    index=False)
allManager_pd = pd.read_csv('./data/manager/allManager_pd.csv',low_memory=False)

In [449]:
# 合并 2 个不同的历史数据，变成新的
manager_history_pd = pd.read_csv('./data/manager/manager_history_pd.csv',low_memory=False)
manager_history_similar_pd = pd.read_csv('./data/manager/manager_history_similar_pd.csv',low_memory=False)
print(manager_history_pd.shape)
print(manager_history_similar_pd.shape)
manager_history_all_pd = pd.merge(
    manager_history_similar_pd,
    manager_history_pd,
    how='left',
    on=['经理ID', '基金代码','基金名称','基金类型','任职天数','任职回报'])

manager_history_all_pd['任职回报'] = manager_history_all_pd['任职回报'].str.strip('%').astype(float)/100
manager_history_all_pd['同类平均'] = manager_history_all_pd['同类平均'].str.strip('%').astype(float)/100
print(manager_history_all_pd.shape)
# print(manager_history_all_pd.head())
manager_history_all_pd.to_csv('./data/manager/manager_history_all_pd.csv', index=False, encoding='utf_8_sig')

(19068, 9)
(19118, 10)
(19118, 13)


In [328]:
# 计算任职时间
def workDay(dayStr):
    manage_wordDay = dayStr.split('又')
    if len(manage_wordDay) > 1:
        manage_word_year = manage_wordDay[0].split('年')[0]
        manage_word_day = manage_wordDay[1].split('天')[0]
        manage_wordDay_Total = int(manage_word_year) * 365 + int(
            manage_word_day)
    else:
        manage_wordDay_Total = dayStr.split('天')[0]
    return manage_wordDay_Total

In [280]:
# 计算相似比率
def similarPer(similarStr):
    x = similarStr.split('|')[0]
    y = similarStr.split('|')[1]
    if x != '-' and y != '-':
        return 1 - int(x)/int(y)
    else:
        return 0
    

In [450]:
# 计算经理数据
manager_history_all_pd = pd.read_csv(
    './data/manager/manager_history_all_pd.csv', low_memory=False)


# manager_history_all_pd['任职同类回报比'] = manager_history_all_pd.apply(
#     lambda x: divide(x['任职回报'],x['同类平均']) , axis=1)

manager_history_all_pd['同类排名比'] = manager_history_all_pd.apply(
    lambda x: similarPer(x['同类排名']) , axis=1)

manager_history_all_pd['任职同类差'] = manager_history_all_pd.apply(
    lambda x: x['任职回报'] - x['同类平均'] , axis=1)

manager_history_all_pd['同类差比和'] = manager_history_all_pd.apply(
    lambda x: x['同类排名比'] + x['任职同类差'] , axis=1)

manager_history_all_pd['同类差比乘'] = manager_history_all_pd.apply(
    lambda x: x['同类排名比'] * x['任职同类差'] , axis=1)

manager_history_all_pd['任职天数'] = manager_history_all_pd.apply(
    lambda x: workDay(x['任职天数']) , axis=1)



# manager_history_all_pd.info()
manager_history_all_pd.to_csv('./data/manager/manager_history_all_1_pd.csv', index=False, encoding='utf_8_sig')

In [451]:
# 把个年内的指标放到总表
allManager_pd = pd.read_csv('./data/manager/allManager_pd.csv',low_memory=False)
mID_s = allManager_pd['manageID']
manager_history_all_pd = pd.read_csv(
    './data/manager/manager_history_all_1_pd.csv', low_memory=False)
for y in range(2, 6, 1):
    print("执行到",y,"年")
    m_history_1_list = []
    m_history_2_list = []
    m_history_3_list = []
    m_history_4_list = []
    for mID in mID_s:
        m_history = manager_history_all_pd.loc[manager_history_all_pd['经理ID'] ==
                                               mID]


        m_history_1 = m_history.loc[m_history['任职天数'] > y * 365]['同类排名比'].sum()
        m_history_1_list.append(m_history_1)
        m_history_2 = m_history.loc[m_history['任职天数'] > y * 365]['任职同类差'].sum()
        m_history_2_list.append(m_history_2)
        m_history_3 = m_history.loc[m_history['任职天数'] > y * 365]['同类差比和'].sum()
        m_history_3_list.append(m_history_3)
        m_history_4 = m_history.loc[m_history['任职天数'] > y * 365]['同类差比乘'].sum()
        m_history_4_list.append(m_history_4)

    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内同类排名比',m_history_1_list)
    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内任职同类差',m_history_2_list)
    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内同类差比和',m_history_3_list)
    allManager_pd.insert(allManager_pd.shape[1],'近' + str(y) + '年内同类差比乘',m_history_4_list)
allManager_pd.to_csv('./data/manager/allManager_1_pd.csv', index=False, encoding='utf_8_sig')

执行到 2 年
执行到 3 年
执行到 4 年
执行到 5 年


In [452]:
# 总表计算各年的总和
allManager_pd = pd.read_csv('./data/manager/allManager_1_pd.csv',low_memory=False)
for y in range(2, 6, 1):
    allManager_pd['近' + str(y) + '年内总分'] = allManager_pd.apply(
        lambda r: 
        r['近' + str(y) + '年内同类排名比'] +
        r['近' + str(y) + '年内任职同类差'] + 
        r['近' + str(y) + '年内同类差比和'] + 
        r['近' + str(y) + '年内同类差比乘'], axis=1)
allManager_pd.to_csv('./data/manager/allManager_2_pd.csv', index=False, encoding='utf_8_sig')

In [536]:
# 查找 N 年内，总分都在前 30%的
top = 0.3
# name = '同类排名比'
# csv_name = 'similar_rank_per'
name_list = ['同类排名比','任职同类差','同类差比和','同类差比乘','总分']
csv_name_list = ['similar_rank_per','office_similar_diff','sum(srp_osd)','mult(srp_osd)','total']

allManager_pd = pd.read_csv('./data/manager/allManager_2_pd.csv',low_memory=False)
for name,csv_name in zip(name_list,csv_name_list):
    allManager_pd_top_2y_pd = allManager_pd.loc[:,['manageID','近2年内' + name]]
    allManager_pd_top_2y_pd.sort_values(by = '近2年内' + name, inplace=True, ascending=False)
    allManager_pd_top_2y_pd = allManager_pd_top_2y_pd.head(round(allManager_pd.shape[0] * top))

    allManager_pd_top_3y_pd = allManager_pd.loc[:,['manageID','近3年内' + name]]
    allManager_pd_top_3y_pd.sort_values(by = '近3年内' + name, inplace=True, ascending=False)
    allManager_pd_top_3y_pd = allManager_pd_top_3y_pd.head(round(allManager_pd.shape[0] * top))

    allManager_pd_top_4y_pd = allManager_pd.loc[:,['manageID','近4年内' + name]]
    allManager_pd_top_4y_pd.sort_values(by = '近4年内' + name, inplace=True, ascending=False)
    allManager_pd_top_4y_pd = allManager_pd_top_4y_pd.head(round(allManager_pd.shape[0] * top))

    allManager_pd_top_5y_pd = allManager_pd.loc[:,['manageID','近5年内' + name]]
    allManager_pd_top_5y_pd.sort_values(by = '近5年内' + name, inplace=True, ascending=False)
    allManager_pd_top_5y_pd = allManager_pd_top_5y_pd.head(round(allManager_pd.shape[0] * top))

    m3 = pd.merge(allManager_pd_top_2y_pd,allManager_pd_top_3y_pd,how='inner',on = 'manageID')
    m4 = pd.merge(m3,allManager_pd_top_4y_pd,how='inner',on = 'manageID')
    m5 = pd.merge(m4,allManager_pd_top_5y_pd,how='inner',on = 'manageID')
    m5.to_csv("./data/manager/allManager_" + csv_name+ "_2y_5y_pd.csv", encoding='utf_8_sig', index=False)
print(m5.shape)

(237, 5)


In [529]:
# 通过基金找经理
fund_similar_top_6m_5y_pd = pd.read_csv('./data/fund/fund_similar_top_6m_5y_pd.csv',low_memory=False)
allManager_pd = pd.read_csv('./data/manager/allManager_2_pd.csv',low_memory=False)
fund_code = fund_similar_top_6m_5y_pd.loc[:,'code']
manager_list = []
# print(allManager_pd.info())
for code in fund_code:
#     print(type(code))
# 找包含基金 code 的经理
    m = allManager_pd[allManager_pd['fundID'].str.contains(str(code))]['manageID'].values[0]
    if m not in manager_list:
        manager_list.append(m)
#     m = allManager_pd.loc[allManager_pd[:,'fundID'].str.contains(code)]
print(len(manager_list))
fund_similar_top_manager_pd = pd.DataFrame(manager_list,columns=['manageID'])


77


In [537]:
csv_name_list = ['similar_rank_per','office_similar_diff','sum(srp_osd)','mult(srp_osd)','total']
for csv_name in csv_name_list:
    allManager_2y_5y_pd = pd.read_csv('./data/manager/allManager_'+csv_name+'_2y_5y_pd.csv',low_memory=False)
    mm = pd.merge(fund_similar_top_manager_pd,allManager_2y_5y_pd,how='inner',on='manageID')
    print(mm.shape)


(29, 5)
(12, 5)
(22, 5)
(21, 5)
(22, 5)


In [539]:
# 通过经理找基金，不理想
allManager_top_2y_5y_pd = pd.read_csv('./data/manager/allManager_top_2y_5y_pd.csv',low_memory=False)
fund_similar_top_6m_5y_pd = pd.read_csv('./data/fund/fund_similar_top_6m_5y_pd.csv',low_memory=False)
allManager_pd = pd.read_csv('./data/manager/allManager_2_pd.csv',low_memory=False)
fund_wide_pd = pd.read_csv('./data/fund/fund_wide_pd.csv',low_memory=False)
fund_wide_pd = fund_wide_pd.loc[:,['code','类型']]
m_id_s = allManager_top_2y_5y_pd.loc[:,'manageID']
code =fund_similar_top_6m_5y_pd.loc[:,'code']
code_list = []
for mID in m_id_s:
    fundID = allManager_pd.loc[allManager_pd['manageID'] == mID]['fundID'].values
    for i in fundID:
        name_list = list(map(int, i.split(',')))
        for j in name_list:
#             print(j)
            if j not in code_list:
                code_list.append(j)
        
# print(len(code_list))
# print(len(code))
m_code = pd.DataFrame(code_list,columns=['code'])
# print(m_code)
code = pd.DataFrame(code,columns=['code'])
# print(code)
# print(m_code.info())
# print(code.info())
mc = pd.merge(code,m_code,how='inner',on = 'code')
mc = pd.merge(mc,fund_wide_pd,how='inner',on = 'code')
print(mc.shape)
mc.to_csv('./mc.csv',encoding='utf_8_sig',index=False)

(43, 2)


In [ ]:
# 测试部分

In [578]:
for t in range(1):

    print(t)

0


In [576]:
import codecs
with open("/Users/hzy/Downloads/wr_s5_np1.json") as f:
     result=json.load(f)
print(type(result))
np1 = np.array(result)
np2 = np1/3
print(np2.shape)
json.dump(np2.tolist(), codecs.open('/Users/hzy/Downloads/wr_s5_np.json', 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True,
          indent=4)

# np.savetxt('/Users/hzy/Downloads/wr_s5_np.json',np2)

<class 'list'>
(1, 4, 2342)


In [486]:
fund_wide_pd = pd.read_csv('./data/fund/fund_wide_pd.csv',low_memory=False)
print(fund_wide_pd.info())
# for code in fund_wide_pd['code']:
#     print(code)
#     print(int(code))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6672 entries, 0 to 6671
Data columns (total 32 columns):
中海可转债债券A                                                                                                                                                                                                                                                                                                                                                           6672 non-null object
000003                                                                                                                                                                                                                                                                                                                                                             6672 non-null int64
债券型                                                                                                                                           

In [464]:
# l1 = eval('[1,2,3,4]')
l1 = eval("['1,2,3,4']")
print(l1,type(l1))
# Out[1]: [1, 2, 3, 4]

['1,2,3,4'] <class 'list'>
